![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/competitions/miad2024-12-prediccion-precio-vehiculos).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import numpy as np

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTest_carListings.zip', index_col=0)

In [4]:
# Visualización datos de entrenamiento
dataTraining.head()

,Price,Year,Mileage,State,Make,Model
0,34995,2017,9913,FL,Jeep,Wrangler
1,37895,2015,20578,OH,Chevrolet,Tahoe4WD
2,18430,2012,83716,TX,BMW,X5AWD
3,24681,2014,28729,OH,Cadillac,SRXLuxury
4,26998,2013,64032,CO,Jeep,Wrangler


In [5]:
# Visualización datos de test
dataTesting.head()

,Year,Mileage,State,Make,Model
ID,,,,,
0,2014,31909,MD,Nissan,MuranoAWD
1,2017,5362,FL,Jeep,Wrangler
2,2014,50300,OH,Ford,FlexLimited
3,2004,132160,WA,BMW,5
4,2015,25226,MA,Jeep,Grand


In [6]:
# Predicción del conjunto de test - acá se genera un número aleatorio como ejemplo
np.random.seed(42)
y_pred = pd.DataFrame(np.random.rand(dataTesting.shape[0]) * 75000 + 5000, index=dataTesting.index, columns=['Price'])
y_pred

,Price
ID,
0,33090.508914
1,76303.572981
2,59899.545636
3,49899.386315
4,16701.398033
...,...
99995,64422.862300
99996,63443.967139
99997,55584.005547


In [7]:
# Guardar predicciones en formato exigido en la competencia de kaggle
y_pred.to_csv('test_submission.csv', index_label='ID')
y_pred.head()

,Price
ID,
0,33090.508914
1,76303.572981
2,59899.545636
3,49899.386315
4,16701.398033


# Entrenamiento de modelos

In [8]:
# Importación de librerías
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV

## Random Forest

In [9]:
# Importación de librerías partículares para este modelo
from sklearn.ensemble import RandomForestRegressor

In [10]:
# Identificación del las columnas por tipo de dato
categorical_columns = ['State', 'Make', 'Model']
numerical_columns = ['Year', 'Mileage']

In [11]:
# Encoding de las columnas
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
    ('num', 'passthrough', numerical_columns)
])

In [12]:
# Pipeline del modelo
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=43))
])

In [13]:
# Partición de los datos en train y test
X_train, X_valid, y_train, y_valid = train_test_split(
    dataTraining.drop('Price', axis=1), dataTraining['Price'], test_size=0.25, random_state=43
)

In [19]:
# Búsqueda de hiperparámetros
param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [10, 20, 30, None],
    'regressor__min_samples_split': [2, 5, 10]
}
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error',n_jobs=1)
grid_search.fit(X_train, y_train)

KeyboardInterrupt: 

In [21]:
# Cálcuo del RMSE
y_pred = grid_search.predict(X_valid)

rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print(f"RMSE: {rmse}")

RMSE: 0.0


In [27]:
# Predicciones en los datos de test
predictions = grid_search.predict(dataTesting)

In [33]:
y_predictions = pd.DataFrame(predictions, index=dataTesting.index, columns=['Price'])


y_predictions.to_csv('test_submission1.csv', index_label='ID')
y_predictions.head()

,Price
ID,
0,21546.992156
1,36494.157561
2,23579.375617
3,7282.012349
4,30688.854940


In [24]:
# Se guarda el modelo
from joblib import dump

dump(grid_search, 'random_forest.joblib')

['random_forest.joblib']

In [ ]:
# Código para cargar el modelo y utilizarlo
from joblib import load

model = load('trained_model.joblib')

predictions = model.predict(X_test)

## XGBoost

In [13]:
# Importación de librerías para el modelo
from xgboost import XGBRegressor

In [14]:
# Pipeline del modelo
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=43))
])

In [15]:
# Definición de la grilla con los hiperparámetros
# Se usa la misma partición que se tenía para el modelo Random Forest
param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [3, 5, 7],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__subsample': [0.8, 1.0]
}

In [16]:
# Búsqueda de hiperparámetros
grid_search = GridSearchCV(xgb_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)

In [17]:
# Entrenamiento del modelo con los hiperparámetros encontrados
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['State',
                                                                          'Make',
                                                                          'Model']),
                                                                        ('num',
                                                                         'passthrough',
                                                                         ['Year',
                                                                          'Mileage'])])),
                                       ('regressor',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     callbacks=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                     devic...
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=43, ...))]),
             param_grid={'regressor__learning_rate': [0.01, 0.1, 0.2],
                         'regressor__max_depth': [3, 5, 7],
                         'regressor__n_estimators': [100, 200],
                         'regressor__subsample': [0.8, 1.0]},
             scoring='neg_mean_squared_error', verbose=1)

In [40]:
# Mejor modelo
best_xgb_model = grid_search.best_estimator_

In [41]:
# Predicciones
y_pred_valid = best_xgb_model.predict(X_valid)

In [42]:
# Cálculo del RMSE
rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
print(f'RMSE: {rmse}')

RMSE: 2242.4386355232814


In [43]:
# Predicciones en los datos de test
y_pred_xgb = best_xgb_model.predict(dataTesting)

In [44]:
y_pred_xgb = pd.DataFrame(y_pred_xgb, index=dataTesting.index, columns=['Price'])


y_pred_xgb.to_csv('test_submission2.csv', index_label='ID')
y_pred_xgb.head()

,Price
ID,
0,21244.152344
1,36787.406250
2,21212.548828
3,8690.245117
4,28708.195312


In [45]:
# Se guarda el modelo
from joblib import dump

dump(best_xgb_model, 'xgb.joblib')

['xgb.joblib']

## Regresión lineal - Lasso

In [18]:
# Importación de librerías para el modelo
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

In [19]:
# Preprocesamiento de los datos
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
        ('num', StandardScaler(), numerical_columns)
    ])

In [20]:
# Pipeline
lasso_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LassoCV(cv=5, random_state=43))
])

In [21]:
# Entrenamiento del modelo
lasso_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['State', 'Make', 'Model']),
                                                 ('num', StandardScaler(),
                                                  ['Year', 'Mileage'])])),
                ('regressor', LassoCV(cv=5, random_state=43))])

In [22]:
# Predicciones en el conjunto de validación
y_pred_lrlasso = lasso_pipeline.predict(X_valid)

In [23]:
# Cálculo del RMSE
rmse = np.sqrt(mean_squared_error(y_valid, y_pred_lrlasso))
print(f'RMSE: {rmse}')

RMSE: 5147.971179298178


In [24]:
y_pred_lrlasso = pd.DataFrame(y_pred_lrlasso, index=dataTesting.index, columns=['Price'])


y_pred_lrlasso.to_csv('test_submission3.csv', index_label='ID')
y_pred_lrlasso.head()

,Price
ID,
0,45669.649661
1,15093.667045
2,34057.770196
3,16879.085236
4,5071.815659


## XGBoost Manual

In [ ]:
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTest_carListings.zip', index_col=0)